In [15]:
#Importing libraries and python files like Luke's Saha file
import saha
import numpy as np
import h5py
import matplotlib.pyplot as plt
import pandas as pd
import scipy.constants as sc

In [16]:
#Placing all data from Skynet into an array
hf = h5py.File("SkyNet_r-process.h5")

OSError: Unable to create file (unable to open file: name = 'SkyNet_r-process.h5', errno = 17, error message = 'File exists', flags = 15, o_flags = c2)

In [6]:
#This section defines temperature, time, and the extended parts of it.
temperature = hf["Temperature"]
time = hf["Time"]
mod_temp = []
full_temp = []
m = (np.log10(temperature[1700]) - np.log10(temperature[1600])) / (np.log10(time[1700]) - np.log10(time[1600]))
for i in range(0,len(temperature)):
    if i < 1700:
        mod_temp.append(np.log10(temperature[i]))
        full_temp.append(temperature[i])
    else:
        del_t = m*(np.log10(time[i]) - np.log10(time[i-1]))
        mod_temp.append(mod_temp[i-1]+del_t)
        full_temp.append(10**mod_temp[i])

NameError: name 'hf' is not defined

In [7]:
## Trying to extract the data for abundances for Samarium
abundance = hf["Y"]
mass = hf["A"]
charge = hf["Z"]
index_list = []
full_abundance = []

for i in range(len(charge)):
    if charge[i] == 62: ##charge of samarium
        index_list.append(i)
        
full_abundance = np.sum(abundance[:, index_list], 1)

NameError: name 'hf' is not defined

In [8]:
#Initializing arrays for density, potentials, Ye
kbEvpGK = 8.617e-5*1e9
density = hf["Density"]
electron_full = np.array(hf["Ye"])
ion_potential = np.array([5.64371,11.078,23.55,41.64,62.7,87,103,118,141,158,179,208,237,257,276,306.5,474,506,543,581,617,658,702,742,782,822,976,1016,1060,1103,1180,1226,1332,1381.6,2371,2466,2569,2676,2782,2887,3028,3137,3253,3363,3669,3766,3873,3971,4227,4337,4548,4655,10494,10762,11060,11337,12264,12588,12992,13262.85,53986.12,55214.23])
Z = 62
f = .3

NameError: name 'hf' is not defined

In [9]:
YI = saha.GetAbundances(full_abundance,np.array(full_temp),np.array(density),np.array(ion_potential))


NameError: name 'full_abundance' is not defined

In [10]:
YI = saha.GetAbundancesFixedYef(full_abundance,np.array(full_temp),np.array(density),np.log(62*full_abundance), np.array(ion_potential))


NameError: name 'full_abundance' is not defined

In [11]:
Ytot = np.sum(YI,1)
for i in range(len(ion_potential)):
    plt.semilogx(kbEvpGK*np.array(full_temp),YI[:,i]/Ytot)
    #plt.plot([ion_potential[i], ion_potential[i]], [0, 1])

plt.ylim([1.e-5, 1])
plt.xlim([0.05, 1e4])
plt.xlabel("Energy(eV)")
plt.ylabel("Abundance")

NameError: name 'YI' is not defined

In [12]:
print (ion_potential)

NameError: name 'ion_potential' is not defined

In [13]:
for xi in ion_potential: 
    plt.semilogx([xi, xi], [0, 1])

NameError: name 'ion_potential' is not defined

In [14]:
np.shape(YI)

NameError: name 'YI' is not defined